In [2]:
#import all libraries 
import pandas as pd
import numpy as np
from lightgbm.sklearn import LGBMClassifier
from sklearn.model_selection import train_test_split



In [69]:
# read the dataset using the compression zip
df_006 = pd.read_csv('data/corona_tested_individuals_ver_006.english.csv.zip',compression='zip')
df_006.head()

c:\python3.7.3\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (1,2,3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2020-04-30,0,0,0,0,0,negative,None,female,Other
1,2020-04-30,1,0,0,0,0,negative,None,female,Other
2,2020-04-30,0,1,0,0,0,negative,None,male,Other
3,2020-04-30,1,0,0,0,0,negative,None,female,Other
4,2020-04-30,1,0,0,0,0,negative,None,male,Other


In [4]:
df_0083 = pd.read_csv('data/corona_tested_individuals_ver_0083.english.csv')
df_0083.head()

c:\python3.7.3\lib\site-packages\IPython\core\interactiveshell.py:3173: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2020-11-12,0,0,0,0,0,negative,No,male,Other
1,2020-11-12,0,1,0,0,0,negative,No,male,Other
2,2020-11-12,0,0,0,0,0,negative,Yes,female,Other
3,2020-11-12,0,0,0,0,0,negative,No,male,Other
4,2020-11-12,0,1,0,0,0,negative,No,male,Contact with confirmed


In [7]:
df_006.dtypes

test_date              object
cough                  object
fever                  object
sore_throat            object
shortness_of_breath    object
head_ache              object
corona_result          object
age_60_and_above       object
gender                 object
test_indication        object
dtype: object

In [8]:
df_006.info

<bound method DataFrame.info of          test_date cough fever sore_throat shortness_of_breath head_ache  \
0       2020-04-30     0     0           0                   0         0   
1       2020-04-30     1     0           0                   0         0   
2       2020-04-30     0     1           0                   0         0   
3       2020-04-30     1     0           0                   0         0   
4       2020-04-30     1     0           0                   0         0   
...            ...   ...   ...         ...                 ...       ...   
278843  2020-03-11     0     0           0                   0         0   
278844  2020-03-11     0     0           0                   0         0   
278845  2020-03-11     0     0           0                   0         0   
278846  2020-03-11     0     0           0                   0         0   
278847  2020-03-11     0     0           0                   0         0   

       corona_result age_60_and_above  gender         t

In [10]:
df_006_train_val = df_006.loc[(df_006['test_date'] >= '2020-03-22') & (df_006['test_date'] <= '2020-03-31')]
df_006_train_val.info

<bound method DataFrame.info of          test_date cough fever sore_throat shortness_of_breath head_ache  \
211278  2020-03-31     0     0           0                   0         0   
211279  2020-03-31     0     0           0                   0         0   
211280  2020-03-31     0     0           0                   0         0   
211281  2020-03-31     0     0           0                   0         0   
211282  2020-03-31     1     0           0                   0         0   
...            ...   ...   ...         ...                 ...       ...   
265116  2020-03-22     1     1           0                   0         0   
265117  2020-03-22     1     1           1                   0         1   
265118  2020-03-22     1     0           0                   0         0   
265119  2020-03-22     1     0           0                   0         0   
265120  2020-03-22     1     0           0                   0         0   

       corona_result age_60_and_above  gender test_indi

In [16]:
df_006['test_date'] = pd.to_datetime(df_006['test_date'], format='%Y-%m-%d')
df_006_train_val = df_006.loc[(df_006['test_date'] >= '2020-03-22') & (df_006['test_date'] <= '2020-03-31')]
df_006_train_val.info

<bound method DataFrame.info of         test_date cough fever sore_throat shortness_of_breath head_ache  \
211278 2020-03-31     0     0           0                   0         0   
211279 2020-03-31     0     0           0                   0         0   
211280 2020-03-31     0     0           0                   0         0   
211281 2020-03-31     0     0           0                   0         0   
211282 2020-03-31     1     0           0                   0         0   
...           ...   ...   ...         ...                 ...       ...   
265116 2020-03-22     1     1           0                   0         0   
265117 2020-03-22     1     1           1                   0         1   
265118 2020-03-22     1     0           0                   0         0   
265119 2020-03-22     1     0           0                   0         0   
265120 2020-03-22     1     0           0                   0         0   

       corona_result age_60_and_above  gender test_indication  
211

In [24]:
nan_in_data = df_006.isna().sum().any()
nan_in_data

False

In [34]:
df_006.columns


Index(['test_date', 'cough', 'fever', 'sore_throat', 'shortness_of_breath',
       'head_ache', 'corona_result', 'age_60_and_above', 'gender',
       'test_indication'],
      dtype='object')

In [70]:
pd.unique(df_006['cough'])


array([0, 1, '0', '1', 'None'], dtype=object)

In [88]:
num_columns = ['cough', 'fever', 'sore_throat', 'shortness_of_breath',
       'head_ache']
for i in num_columns:
    df_006 = df_006[df_006[i] != 'None']
    print (pd.unique(df_006[i]))
df_006[num_columns] = df_006[num_columns].apply(pd.to_numeric)
df_006[num_columns] .dtypes

[0 1]
[0 1]
[0 1]
[0 1]
[0 1]


cough                  int64
fever                  int64
sore_throat            int64
shortness_of_breath    int64
head_ache              int64
dtype: object

In [89]:
df_006.info

<bound method DataFrame.info of          test_date  cough  fever  sore_throat  shortness_of_breath  head_ache  \
0       2020-04-30      0      0            0                    0          0   
1       2020-04-30      1      0            0                    0          0   
2       2020-04-30      0      1            0                    0          0   
3       2020-04-30      1      0            0                    0          0   
4       2020-04-30      1      0            0                    0          0   
...            ...    ...    ...          ...                  ...        ...   
278843  2020-03-11      0      0            0                    0          0   
278844  2020-03-11      0      0            0                    0          0   
278845  2020-03-11      0      0            0                    0          0   
278846  2020-03-11      0      0            0                    0          0   
278847  2020-03-11      0      0            0                    0          0

In [92]:
cat_columns = ['corona_result', 'age_60_and_above', 'gender',
       'test_indication']

In [93]:
for i in cat_columns:
    df_006 = df_006[df_006[i] != 'None']
    print (pd.unique(df_006[i]))

['negative' 'positive' 'other']
['Yes' 'No']
['male' 'female']
['Other' 'Abroad' 'Contact with confirmed']


In [95]:
df_006 = df_006[df_006['corona_result'] != 'other']

In [96]:
df_006_train_val = df_006.loc[(df_006['test_date'] >= '2020-03-22') & (df_006['test_date'] <= '2020-03-31')]
df_006_train_val.info

<bound method DataFrame.info of          test_date  cough  fever  sore_throat  shortness_of_breath  head_ache  \
211278  2020-03-31      0      0            0                    0          0   
211279  2020-03-31      0      0            0                    0          0   
211280  2020-03-31      0      0            0                    0          0   
211281  2020-03-31      0      0            0                    0          0   
211282  2020-03-31      1      0            0                    0          0   
...            ...    ...    ...          ...                  ...        ...   
265116  2020-03-22      1      1            0                    0          0   
265117  2020-03-22      1      1            1                    0          1   
265118  2020-03-22      1      0            0                    0          0   
265119  2020-03-22      1      0            0                    0          0   
265120  2020-03-22      1      0            0                    0          0